In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
import warnings
import re
warnings.filterwarnings("ignore")

In [85]:
def remove_html_tags(text):
    clean = re.compile(r'<[^>]+>|// Page.+')
    return clean.sub('', text)

In [104]:
def read_wissports_page(category: str, page_num : int):
    base_url = "https://www.wissports.net/stats/tab_league_instance_player_stats/211430.js"
    csrf_token = "xRU1eQ9QscQ0GM6oGPQ//8nzIINTwXV+14qa5X11A2+NDnD3jPItvNeGVm9iraNYKGd0H8xdQlq0eZ6AkNfNfQ=="
    page_number = page_num
    
    # Define the query parameters based on your fetch command
    params = {
        "dir": "desc",
        "order_by": "rshyd",
        "page": page_number,  # This is the key parameter for pagination
        "stat_module": f"football_{category}",
        "subseason": "917849",
        "tag_match": "all",
        "tags[]": "2"
    }
    
    # Define the headers, including the essential X-CSRF-Token
    headers = {
        "accept": "*/*;q=0.5, text/javascript, application/javascript, application/ecmascript, application/x-ecmascript",
        "x-csrf-token": csrf_token,  # Use the token you found in the network tab
        "x-requested-with": "XMLHttpRequest",
        # Adding a User-Agent is always a good idea
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    print(f"Fetching page {page_number}...")
    
    response = requests.get(base_url, params=params, headers=headers)
    
    bs = BeautifulSoup(response.text)
    raw_html = str(bs.find("table"))
    
    html_strio = StringIO(raw_html.replace("\\n", "").replace('<\/t.+>', ''))
    
    df = pd.read_html(html_strio)[0]
    df.columns = list(pd.Series(df.columns).apply(remove_html_tags))
    for col in df.columns:
        df[col] = df[col].apply(remove_html_tags)
    df.iloc[-1, -1] = df.iloc[-1, -1].split(" ")[0]
    return df

In [106]:
read_wissports_page("passing", 1)

Fetching page 1...


,#,Name,Team,G,Comp,Att,Comp %,Yds,Yds/Att,TD,Pass YPG,Int,Rate
0,3,Patrick Greisen,WDP,12,249,352,70.7,3846,10.9,49,320.5,9,135.5
1,15,Malcolm MacGrath,ASHW,10,165,277,59.6,2402,8.7,21,240.2,5,105.6
2,2,Max Raimer,SUNPE,11,171,245,69.8,2339,9.5,21,212.6,5,120.1
3,19,Michael Thiede,SLI,14,123,199,61.8,2113,10.6,23,150.9,6,123.8
4,17,Bryce Ott,OSH-N,9,117,193,60.6,2088,10.8,25,232.0,6,124.3
5,6,Bron Schaefer,KAUK,12,148,247,59.9,2036,8.2,18,169.7,9,95.5
6,9,Brady Johnson,MEN,10,143,247,57.9,1856,7.5,21,185.6,11,91.4
7,10,Alex Evanoff,CED,10,129,208,62.0,1807,8.7,13,180.7,9,92.8
8,5,Connor Navis,A-WEST,9,138,224,61.6,1792,8.0,19,199.1,13,90.8
9,12,Nick Stellrecht,NEWRCH,12,124,196,63.3,1750,8.9,22,145.8,9,110.3
